# General Analysis - Magnetic Rayleigh-Taylor Instability

### Toward a full analysis of MRT instability (This includes Hall Effects) - Simple Plasma class for caclualting the relevant parameters


'J.D. Huba, Rayleigh-Taylor Instability: Comparison of Hybrid and Nonideal MHD simulation, 1998'

'R. K. Bera, Yang Song, The effect of viscosity and resistivity on Rayleigh-Taylor instability induced mixing in magnetised HEDP, 2021'

'F.R.S. Taylor, The instability of liquid surfaces when accelerated in a direction perpendicular to their planes, 1949'

In [ ]:
# Libraries

import numpy as np
import matplotlib.pyplot as plt
import scipy.constants as cons
import matplotlib.gridspec as gridspec

from scipy.interpolate import CubicSpline


# Function to return ZTe relation from ionisation state tables



In [ ]:
class plasma:
    def __init__(self, A, ne, Te, V, L, Atw):
        '''
        Args:
            example:
            ---------------------------------------------------------------------------------------
            al_flow = {'A':27, 'ne': 6.8e18, 'Te': Te , 'V':4e6,  'L':0.2, 'Atw': 1, 'Zsrc': 'exp'}
            al=Plasma(**al_flow)
            ---------------------------------------------------------------------------------------
            A:      ion mass in nucleon masses
            Te      electron temperature in eV
            V:      velocity in cm/s
            L:      length scale shock in cm.
            Atw:    Atwood Number
            Zsrc:   Charge State Model source, 'exp' or 'FLY'

        '''
        self.A      =   A
        self.ne     =   ne
        self.Te     =   Te
        self.V      =   V
        self.L      =   L
        self.Atw    =   Atw
        self.Zsrc   =   Zsrc    

        # Estimate Ionisation Charge State - Z - from Tabled Values
        self.ZTe()
        self.Z = Z_mod(self.Te)

        # Calculate Plasma Parameters
        self.params()
        # Calculate MRT Parameters
        self.MRTparams()
    
    def ZTe(self):
        A = self.A
        source = self.Zsrc
        # source = 'FLY'
        if A == 27:     # Aluminium
            if source == 'FLY':
                T_e, Z =  np.genfromtxt('Al_Z_1e18.txt', delimiter = '  ', usecols = [0,1], unpack = True)
            else:
                T_e, Z =  np.genfromtxt('zb_Al.dat', delimiter=' ', skip_header=2, usecols = [0,3], unpack = True)
        
            Z_mod = CubicSpline(T_e, Z)
            return Z_mod
        if A == 64:     # Copper
            if source == 'FLY':
                T_e, Z = np.genfromtxt('Cu_Z_1e18.txt', delimiter='  ', usecols = [0,1], unpack = True)
            else:
                T_e, Z = np.genfromtxt('zb_Cu.dat', delimiter='     ', skip_header=2, usecols = [0,2], unpack = True)
            Z_mod = CubicSpline(T_e, Z)
            return Z_mod
        if A == 108:    # Silver
            if source == 'FLY':
                T_e, Z = np.genfromtxt('Ag_Z_1e18.txt', delimiter='  ', usecols = [0,1],unpack = True)
            else:
                print("No data available for A:",A)
            Z_mod = CubicSpline(T_e, Z)
            return Z_mod
        if A == 183:    # Tungsten
            if source == 'FLY':
                T_e, Z = np.genfromtxt('W_Z_1e18.txt', delimiter='  ', usecols = [0,1],unpack = True)
            else:
                T_e, Z = np.genfromtxt('zb_W.dat', delimiter=' ', usecols = [0,1],unpack = True)
            Z_mod = CubicSpline(T_e, Z)
            return Z_mod
        else:
            print("No data available for A:", A)

    def params(self):
        m_e=cons.m_e            # Electron Mass
        e=cons.e                # Elemental Charge
        mu_0=cons.mu_0          # Permeability Vacuum
       
        # Calculate Coulomb Log
        self.CoulombLog()

        # Acceleration
        self.a      =    self.V**2/self.L                                               # Acceleration [cm s^-2]
        # Collision frequency
        self.nu_ei  =    2.91e-6*self.Z*self.ne*self.col_log_ei*self.Te**-1.5           # Collision Frequency Electrons - Ions
        # magnetic diffusivity
        n_e = self.ne * 1e6     # Electron Density [m-3]
        self.sigma  =    n_e*e**2/(m_e*self.nu_ei)                                      # Conductivity [s kg^-3 m^-3 C^-3]
        self.Dm     =    1/(self.sigma*mu_0)                                            # Magnetic Diffusivity [m^2 s^-1]
        self.Dm     =    self.Dm*1e4                                                    # m^2 s^-1 --> cm^2 s^-1

    def CoulombLog(self):
        # Couloumb log - NRL
        T_e = self.Te
        n_e = self.ne

        self.col_log_ei = 23-np.log(n_e**0.5*self.Z*T_e**-1.5)

    def MRTparams(self):
        # calculate Wave Number
        self.k      =    ((self.Atw*self.a)**(1/3))*(self.Dm)**(-2/3)      
        # Calculate Wavelength
        self.lmax   =    2*np.pi/(self.k)             # maximum lambda max in cm
        # Calculate Growth rate
        self.gr     =    (self.Atw*self.a*self.k)**0.5     # maximum growth rate in 1/s
        # Calculate Tau
        self.tau    =    (1/self.gr)*1e9          # maximum growth time in ns

 ### Magnetic diffusivity is calculated from the NRL's  Formulas
 ### <h1><center> $D_{m} = {\eta}/{\mu_{0}}$  ${m^2} s^{-1}$ </center></h1>